In [1]:
import pandas as pd

df = pd.read_csv('exams.csv')
df.columns = [c.replace(' ','_') for c in df.columns]
originalFeatures = df.columns
print('originalFeatures count', len(originalFeatures))
print('originalFeatures', originalFeatures)
print(df.head())

In [2]:
df.isnull().sum()

In [3]:
df.describe()

In [4]:
df.test_preparation_course.value_counts()

In [5]:
335/665 #the data is slightly imbalanced but it's ok.

In [6]:
X = df[['reading_score','math_score']]
y = df.test_preparation_course

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[-0.13591401,  0.03923309],
       [-0.67902695, -1.714564  ],
       [-0.61113783, -0.48041049]])

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [9]:
X_train.shape

(750, 2)

In [10]:
X_test.shape

(250, 2)

In [11]:
y_train.value_counts()

none         499
completed    251
Name: test_preparation_course, dtype: int64

In [12]:
251/499

0.503006012024048

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=10)
scores

array([0.62, 0.66, 0.51, 0.55, 0.63, 0.54, 0.52, 0.58, 0.49, 0.6 ])

In [14]:
scores.mean()

0.57

In [15]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(base_estimator = DecisionTreeClassifier(),
                 n_estimators = 100,
                  max_samples=0.8,
                  oob_score=True,
                  random_state=0
                 )
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.5666666666666667

In [16]:
bag_model.score(X_test, y_test)

0.616

In [17]:
bag_model = BaggingClassifier(base_estimator = DecisionTreeClassifier(),
                 n_estimators = 100,
                  max_samples=0.8,
                  oob_score=True,
                  random_state=0
                 )

cross_val_score(bag_model, X, y, cv=10)
scores.mean()

0.57

In [18]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(), X, y, cv = 10)
scores.mean()

0.6050000000000001

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
weak_learner = DecisionTreeClassifier(max_depth = 3)
clf = AdaBoostClassifier(base_estimator=weak_learner, n_estimators=100)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: ", accuracy)

Accuracy:  0.6
